In [ ]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
# Importing libraries
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import seaborn as sns
from collections import Counter
from numpy.random import randn
from datetime import datetime

from geopy.distance import vincenty
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
tqdm.pandas()
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
from scipy import stats
from dateutil import parser
import eli5
from eli5.sklearn import PermutationImportance
from skopt.space import Real
import warnings
warnings.filterwarnings("ignore")





import matplotlib.pyplot as plt
%matplotlib inline





# Change pandas viewing options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Gathering Data
sydney_listings = pd.read_csv('sydney-2016-01-03-listings.csv')
sydney_reviews = pd.read_csv('sydney-2016-01-03-reviews.csv')
sydney_calendar = pd.read_csv('sydney-2016-01-03-calendar.csv')

melbourne_listings = pd.read_csv('melbourne-2016-01-03-listings.csv')
melbourne_reviews = pd.read_csv('melbourne-2016-01-03-reviews.csv')
melbourne_calendar = pd.read_csv('melbourne-2016-01-03-calendar.csv')

In [ ]:
melbourne_listings.shape, sydney_listings.shape

In [ ]:
melbourne_reviews.shape, sydney_reviews.shape

In [ ]:
melbourne_calendar.shape, sydney_calendar.shape

In [ ]:
all_listings = pd.concat([sydney_listings, melbourne_listings],
                        keys = [1, 2])
all_listings.head(2)
all_listings.reset_index(level=0, inplace=True)
all_listings.rename(columns={'level_0': 'city_index'}, inplace=True)

In [ ]:

all_listings.head()
all_listings.shape

In [ ]:
all_reviews = pd.concat([sydney_reviews, melbourne_reviews],
                        keys = [1, 2])
all_reviews.head(2)
all_reviews.reset_index(level=0, inplace=True)
all_reviews.rename(columns={'level_0': 'city_index'}, inplace=True)

In [ ]:
all_reviews.shape

In [ ]:
all_calendar = pd.concat([sydney_calendar, melbourne_calendar],
                        keys = [1, 2])
all_calendar.head(2)
all_calendar.reset_index(level=0, inplace=True)
all_calendar.rename(columns={'level_0': 'city_index'}, inplace=True)

In [ ]:
all_calendar.shape

In [ ]:
all_listings.info()

In [ ]:
unique_nums = all_listings.nunique()
masked_columns = [col for col in all_listings.columns if unique_nums[col] == 0]
masked_columns

In [ ]:
Counter(all_listings.license.dropna())

In [ ]:
masked_columns += ['listing_url', 
                   'scrape_id', 
                   'picture_url',
                   'medium_url',
                   'host_url',
                   'thumbnail_url',
                   'xl_picture_url',
                   'host_thumbnail_url', 
                   'host_picture_url']

In [ ]:

all_listings.head(5)

In [ ]:
all_listings.drop(masked_columns, axis=1, inplace=True)

In [ ]:
all_listings.head(5)

In [ ]:
price_cols = ['price', 
              'weekly_price', 
              'monthly_price', 
              'security_deposit', 
              'cleaning_fee', 
              'extra_people']

In [ ]:
all_listings.head(1).price.values

In [ ]:
sl = '$60.00'

In [ ]:
def prices_to_numbers(price_string):
    '''
    Converts USD prices from string to numeric format
    
    Args:
        price_string (string): USD price in string format (e.g., '$123,456.00')
    
    Returns:
        price_numeric (float): USD price in numeric format (e.g., 123456.00)
    '''
    
    price_numeric = float(str(price_string).replace(',', '').split('$')[-1])
    return price_numeric

prices_to_numbers(s1)

In [ ]:
for col in price_cols:
    print(col)
    all_listings[col] = all_listings[col].apply(prices_to_numbers)

In [ ]:
all_listings.select_dtypes(float).head(2)

In [ ]:

all_listings.head(5)

In [ ]:
boolean_cols = ['host_is_superhost', 
               'host_has_profile_pic',
               'host_identity_verified',
               'is_location_exact',
               'has_availability',
               'requires_license',
               'instant_bookable',
               'require_guest_profile_picture',
               'require_guest_phone_verification']

In [ ]:
def booleans_to_numbers(s):
    '''
    Converts "first letter boolean" strings to integers
    
    Args:
        bool_string (string): 't', 'f' or other
    
    Returns:
        bool_number (int): 1, 0 or None
    '''
    
    
    if s == 'f':
        return 0
    elif s == 't':
        return 1
    return None

In [ ]:
for col in boolean_cols:
    print(col)
    all_listings[col] = all_listings[col].apply(booleans_to_numbers)

In [ ]:
all_listings.select_dtypes(object).head(2)

In [ ]:
all_listings.to_csv('all_listings.csv', encoding='utf-8', index=False)

In [ ]:
all_reviews.info()

In [ ]:
all_reviews.head(5)

In [ ]:
all_listings.summary.head(50)

In [ ]:
all_listings.description.head(50)

In [ ]:
all_listings.notes.head(50)

In [ ]:
all_listings.property_type.unique()

In [ ]:
all_listings.bed_type.unique()

In [ ]:
all_listings.cancellation_policy.unique()

In [ ]:
all_listings.room_type.unique()

In [ ]:
#room_type_dict = {'Shared room': 1, 'Private room': 2, 'Entire home/apt': 3}
#all_listings.room_type.replace(room_type_dict, inplace=True)

In [ ]:
all_listings.amenities.head().values

In [ ]:
#all_listings.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
#Finally, we want to add geographical distance from cities to city centers
lat_center_dict = {1: -33.8678500, 2: 37.8140000}
lon_center_dict = {1: 151.2073200, 2: 144.9633200}

In [ ]:
all_listings['lat_center'] = all_listings.city_index.replace(lat_center_dict)
all_listings['lon_center'] = all_listings.city_index.replace(lon_center_dict)

In [ ]:
%%time
# calculates Vincenty distance from the city center using https://en.wikipedia.org/wiki/Vincenty%27s_formulae 
all_listings['distance_center'] = all_listings.apply(lambda x: vincenty((x['latitude'], x['longitude']), (x['lat_center'], x['lon_center'])).km, axis = 1)

In [ ]:
all_listings['distance_center'].describe()

In [ ]:
all_listings.shape

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(all_listings.corr());
plt.show()

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
#Positive and negative reviews

In [ ]:
test_sentence_1 = 'Room was nice. could have been a lot worse. I really liked it.'
test_sentence_2 = 'I detest this room. It was terrible.'

# calculates compound sentiment polarity of the sentence
vader_polarity_compound = lambda s: (SentimentIntensityAnalyzer().polarity_scores(s))['compound']

vader_polarity_compound(test_sentence_1), vader_polarity_compound(test_sentence_2)

In [63]:
sydney_reviews.date = sydney_reviews.date.apply(lambda x: pd.to_datetime(x))

array(['within an hour', 'within a few hours', 'within a day', nan,
       'a few days or more'], dtype=object)

AttributeError: 'DataFrame' object has no attribute 'house_rules'

The plots above show that the distrubution of the scores is skewed and most of the reviews are positive (scores are in the scale from 0 to 100)

We will start with something fun, like building a wordmap from the comments

In [ ]:
from langdetect import detect
def lang_detection(series):
    isEn = []
    for text in series:
        try:
            language = detect(text)
        except:
            language = "error"
            print("This row throws and error:", text)
        
        if language == 'en':
            isEn.append(True)
        else:
            isEn.append(False)
            
    return isEn
#     return [True if detect(text) == 'en' else False for text in series]

In [ ]:

lang_isen = lang_detection(reviews_df.comments)

In [ ]:
reviews_df = reviews_df[lang_isen]